In [ ]:
import torch
import torch.nn as nn

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, act="relu", use_dropout=False):
        super(Block, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False, padding_mode="reflect")
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU() if act == "relu" else nn.LeakyReLU(0.2),
        )
        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)
        self.down = down

    def forward(self, x):
        x = self.conv(x)
        return self.dropout(x) if self.use_dropout else x

class InnerUNet(nn.Module):
    def __init__(self, in_channels, features):
        super(InnerUNet, self).__init__()
        self.down1 = Block(in_channels, features * 4, down=True, act="leaky", use_dropout=False)
        self.down2 = Block(features * 4, features * 8, down=True, act="leaky", use_dropout=False)
        self.down3 = Block(features * 8, features * 16, down=True, act="leaky", use_dropout=False)
        self.bottleneck = nn.Sequential(
            nn.Conv2d(features * 16, features * 32, 4, 2, 1), nn.ReLU()
        )
        self.up1 = Block(features * 32, features * 16, down=False, act="relu", use_dropout=False)
        self.up2 = Block(features * 16, features * 8, down=False, act="relu", use_dropout=False)
        self.up3 = Block(features * 8, features * 4, down=False, act="relu", use_dropout=False)
        self.final_up = nn.ConvTranspose2d(features * 4, in_channels, kernel_size=4, stride=2, padding=1)

    def forward(self, x):
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        bottleneck = self.bottleneck(d3)
        up1 = self.up1(bottleneck)
        up2 = self.up2(torch.cat([up1, d3], dim=1))
        up3 = self.up3(torch.cat([up2, d2], dim=1))
        final_output = self.final_up(torch.cat([up3, d1], dim=1))  
        
        return bottleneck, up1, up2, up3, final_output

# Outer U-Net (with depth 7) integrating the Inner U-Net
class OuterUNet(nn.Module):
    def __init__(self, in_channels, features):
        super(OuterUNet, self).__init__()
        self.initial_down = nn.Sequential(
            nn.Conv2d(in_channels, features, 4, 2, 1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),
        )
        self.down1 = Block(features, features * 2, down=True, act="leaky", use_dropout=False)
        self.down2 = Block(features * 2, features * 4, down=True, act="leaky", use_dropout=False)
        self.down3 = Block(features * 4, features * 8, down=True, act="leaky", use_dropout=False)
        self.down4 = Block(features * 8, features * 8, down=True, act="leaky", use_dropout=False)
        self.down5 = Block(features * 8, features * 8, down=True, act="leaky", use_dropout=False)
        self.down6 = Block(features * 8, features * 8, down=True, act="leaky", use_dropout=False)

        self.inner_unet = InnerUNet(features * 2, features // 2)  # Inner U-Net starts here

        self.bottleneck_outer = nn.Sequential(
            nn.Conv2d(features * 8, features * 8, 4, 2, 1), nn.ReLU()
        )

        self.up1 = Block(features * 8 * 2, features * 8, down=False, act="relu", use_dropout=True)
        self.up2 = Block(features * 8 * 2, features * 8, down=False, act="relu", use_dropout=True)
        self.up3 = Block(features * 8 * 2, features * 4, down=False, act="relu", use_dropout=False)
        self.up4 = Block(features * 4 * 2, features * 2, down=False, act="relu", use_dropout=False)
        self.up5 = Block(features * 2 * 2, features, down=False, act="relu", use_dropout=False)
        self.final_up = nn.Sequential(
            nn.ConvTranspose2d(features * 2, in_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        d1 = self.initial_down(x)
        d2 = self.down1(d1)
        d3 = self.down2(d2)
        d4 = self.down3(d3)
        d5 = self.down4(d4)
        d6 = self.down5(d5)
        d7 = self.down6(d6)

        # Inner U-Net connections
        bottleneck_inner, up1_inner, up2_inner, up3_inner, final_output_inner = self.inner_unet(d1)

        # Outer U-Net continues from bottleneck with inner-outer concatenations
        bottleneck_outer = self.bottleneck_outer(d7)
        up1 = self.up1(torch.cat([bottleneck_outer, bottleneck_inner, d7], dim=1))
        up2 = self.up2(torch.cat([up1_inner ,up1, d6], dim=1))
        up3 = self.up3(torch.cat([up2, d5], dim=1))
        up4 = self.up4(torch.cat([up2_inner, up3, d4], dim=1))
        up5 = self.up5(torch.cat([up4, d3], dim=1))
        up6 = self.final_up(torch.cat([up3_inner, up5, d2], dim=1))
        return up6

def test_model():
    x = torch.randn((1, 3, 256, 256))  
    model = OuterUNet(in_channels=3, features=64)
    preds = model(x)
    print(preds.shape)

test_model()